Plot snapshot of depth-averaged KE.

In [1]:
import numpy as np
import xarray as xr
import cmocean as cmocean
import numpy.linalg as la
from scipy.io import netcdf
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
from xgcm import Grid
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib
from matplotlib.colors import SymLogNorm, BoundaryNorm, LogNorm
from matplotlib.ticker import LogFormatter
from matplotlib.ticker import MaxNLocator
import matplotlib.ticker as ticker

mpl.rcParams['text.usetex'] = True
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = ['Computer Modern Roman']

In [2]:
def get_colors_from_cmap(cmap_name, n_colors):
    cmap = cm.get_cmap(cmap_name)
    return [cmap(i / (n_colors - 1)) for i in range(n_colors)]

blues   = get_colors_from_cmap('Blues', 10)
reds = ['#f69a8b', '#b91a38']
greys = get_colors_from_cmap('Greys', 10)

colorlist_p5 = [blues[4], reds[0]]
colorlist_p25 = [blues[8], reds[1]]
colorlist_p03125 = [greys[3], greys[5], 'k']

/state/partition1/job-65870369/ipykernel_2491352/3569882092.py:2: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap(cmap_name)


In [3]:
# Snapshot
root = '/scratch/mp6191/NW2_TracerBackscatter'

exp = '/p5_noBS_KHTR0_2'
p5_noBS = xr.open_mfdataset(root + exp + '/snapshots*.nc', decode_times = False)#.isel(time = -1)
p5_noBS_h = xr.open_mfdataset(root + exp + '/MMT_fields*.nc', decode_times = False).h.mean('time')
p5_static = xr.open_dataset(root + exp + '/static.nc', decode_times = False)
p5_eta = xr.open_dataset(root + exp + '/MOM_IC.nc', decode_times = False)

exp = '/p5_SQGBS_KHTR0_2'
p5_SQGBS = xr.open_mfdataset(root + exp + '/snapshots*.nc', decode_times = False)#.isel(time = -1)
p5_SQGBS_h = xr.open_mfdataset(root + exp + '/MMT_fields*.nc', decode_times = False).h.mean('time')

exp = '/p25_noBS_2'
p25_noBS = xr.open_mfdataset(root + exp + '/snapshots*.nc', decode_times = False)#.isel(time = -1)
p25_noBS_h = xr.open_mfdataset(root + exp + '/MMT_fields*.nc', decode_times = False).h.mean('time')
p25_static = xr.open_dataset(root + exp + '/static.nc', decode_times = False)
p25_eta = xr.open_dataset(root + '/p25_SpinUp' + '/MOM_IC.nc', decode_times = False)

exp = '/p25_SQGBS_2'
p25_SQGBS = xr.open_mfdataset(root + exp + '/snapshots*.nc', decode_times = False)#.isel(time = -1)
p25_SQGBS_h = xr.open_mfdataset(root + exp + '/MMT_fields*.nc', decode_times = False).h.mean('time')

exp = '/p03125_2'
p03125 = xr.open_mfdataset(root + exp + '/snapshots*.nc', decode_times = False).isel(time = slice(None, 180, 2))#.isel(time = -1)
p03125_h = xr.open_mfdataset(root + exp + '/MMT_fields*.nc', decode_times = False).h.mean('time')
p03125_static = xr.open_dataset(root + exp + '/static.nc', decode_times = False)

# Grid
p5_grid = Grid(p5_noBS, coords = {'X': {'center': 'xh', 'outer': 'xq'},
                                  'Y': {'center': 'yh', 'outer': 'yq'}})

p25_grid = Grid(p25_noBS, coords = {'X': {'center': 'xh', 'outer': 'xq'},
                                    'Y': {'center': 'yh', 'outer': 'yq'}})

p03125_grid = Grid(p03125, coords = {'X': {'center': 'xh', 'outer': 'xq'},
                                     'Y': {'center': 'yh', 'outer': 'yq'}})

/ext3/miniconda3/lib/python3.10/site-packages/xarray/core/concat.py:527: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
/ext3/miniconda3/lib/python3.10/site-packages/xarray/core/concat.py:527: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
/ext3/miniconda3/lib/python3.10/site-packages/xarray/core/concat.py:527: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
/ext3/miniconda3/lib/python3.10/site-packages/xarray/core/concat.py:527: FutureWarning: unique with argument that is not not a Ser

In [4]:
def depth_integrate_EKE(ds, grid):

    u = grid.interp(ds.u, axis = 'X')
    v = grid.interp(ds.v, axis = 'Y')

    uprime = (u.isel(time = 49) - u.mean('time'))
    vprime = (v.isel(time = 49) - v.mean('time'))

    h = ds.h.isel(time = 49)

    EKE = (uprime ** 2 + vprime ** 2) / 2
    EKE_int = (h * EKE).sum('zl') / h.sum('zl')

    return EKE_int.load()

In [5]:
p5_noBS_KE = depth_integrate_EKE(p5_noBS, p5_grid)
p5_SQGBS_KE = depth_integrate_EKE(p5_SQGBS, p5_grid)
p25_noBS_KE = depth_integrate_EKE(p25_noBS, p25_grid)
p25_SQGBS_KE = depth_integrate_EKE(p25_SQGBS, p25_grid)

/ext3/miniconda3/lib/python3.10/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/ext3/miniconda3/lib/python3.10/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/ext3/miniconda3/lib/python3.10/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/ext3/miniconda3/lib/python3.10/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


In [ ]:
p03125_KE = depth_integrate_EKE(p03125, p03125_grid)

In [ ]:
def time_depth_zonal_ave_EKE(ds, ds_h):
    grid = Grid(ds, coords = {'X': {'center': 'xh', 'outer': 'xq'},
                              'Y': {'center': 'yh', 'outer': 'yq'}})
    
    u = ds.u
    v = ds.v

    up = u - u.mean('time')
    vp = v - v.mean('time')

    up = grid.interp(up, axis = 'X')
    vp = grid.interp(vp, axis = 'Y')
    
    EKE = 0.5 * (up ** 2 + vp ** 2)
    EKE_t_av = EKE.mean('time')
    EKE_t_d_av = (EKE_t_av * ds_h).sum('zl') / ds_h.sum('zl')
    EKE_t_d_z_av = EKE_t_d_av.mean('xh')

    return EKE_t_d_z_av.load()

In [ ]:
p5_noBS_EKE_zm = time_depth_zonal_ave_EKE(p5_noBS.sel(time = slice(43000, 43500)), p5_noBS_h)
p5_SQGBS_EKE_zm = time_depth_zonal_ave_EKE(p5_SQGBS.sel(time = slice(43000, 43500)), p5_SQGBS_h)

p25_noBS_EKE_zm = time_depth_zonal_ave_EKE(p25_noBS.sel(time = slice(43000, 43500)), p25_noBS_h)
p25_SQGBS_EKE_zm = time_depth_zonal_ave_EKE(p25_SQGBS.sel(time = slice(43000, 43500)), p25_SQGBS_h)

In [ ]:
p03125_EKE_zm = xr.open_dataset('/scratch/mp6191/NW2_TracerBackscatter/MiscFields/p03125_EKE_zm.nc', decode_times = False)
p03125_EKE_zm = p03125_EKE_zm[list(p03125_EKE_zm.data_vars)[0]].load()

### Energy-containing scale

In [ ]:
Re = 6.37e6

def thickness_to_eta(ds, static, eta):
    '''
    Converts thickness to eta
    '''
    xh = ds.xh.values
    yh = ds.yh.values
    zi = eta.Interface.values
    time = ds.time.values
    shape = (time.size, zi.size, yh.size, xh.size)
    e = xr.DataArray(data = np.zeros(shape),
                     dims = ['time', 'zi', 'yh', 'xh'], 
                     coords = {'xh' : xh, 'yh' : yh, 'zi' : zi, 'time' : time}).rename('eta')

    D = static.depth_ocean.load()
    h = ds.h.load()
    for k in range(zi.size):
        e[:, k, :, :] = h.isel(zl = slice(k, None)).sum('zl', skipna = False) - D

    return e

def calc_length(ds, static, eta):
    '''
    Calculates energy containing scale
    '''

    ssh = thickness_to_eta(ds, static, eta).isel(zi = 0)
    ssh_clim = ssh.mean('time')
    ssh_anom = ssh - ssh_clim

    lat = static['geolat']
    ssh_anom_dx = ssh_anom.differentiate('xh') * 360 / (2 * np.pi * Re * np.cos(lat * np.pi / 180))
    ssh_anom_dy = ssh_anom.differentiate('yh') * 360 / (2 * np.pi * Re)

    le = np.sqrt((ssh_anom ** 2).mean('time') / (ssh_anom_dx ** 2 + ssh_anom_dy ** 2).mean('time')).rename('Le')

    return le.load()

In [ ]:
exp = '/p5_noBS_KHTR0_2'
p5_eta = xr.open_dataset(root + exp + '/MOM_IC.nc', decode_times = False)
p25_eta = xr.open_dataset(root + '/p25_SpinUp' + '/MOM_IC.nc', decode_times = False)

In [ ]:
p5_noBS_le = calc_length(p5_noBS, p5_static, p5_eta).load()
p5_SQGBS_le = calc_length(p5_SQGBS, p5_static, p5_eta).load()
p25_noBS_le = calc_length(p25_noBS, p25_static, p25_eta).load()
p25_SQGBS_le = calc_length(p25_SQGBS, p25_static, p25_eta).load()
p03125_le = xr.open_dataset(root + '/p03125_2' + '/EddyLength.nc')['Le'].load()

In [ ]:
# Grid spacing from Hallberg 2013
p5_dx = np.sqrt((p5_static.dxCu.mean('xq') ** 2 + p5_static.dyCu.mean('xq') ** 2) / 2)
p25_dx = np.sqrt((p25_static.dxCu.mean('xq') ** 2 + p25_static.dyCu.mean('xq') ** 2) / 2)

### Plot

In [ ]:
fig = plt.figure(figsize = (10.5, 7), dpi = 300)
fontsize = 13
linewidth = 1.25

# Main grid
main_gs = gridspec.GridSpec(2, 2, width_ratios = [5, 0.1], height_ratios = [1, 0.6], hspace = 0.375, wspace = 0.05, figure = fig)
# Top grid
top_gs = gridspec.GridSpecFromSubplotSpec(1, 5, width_ratios = [1, 1, 1, 1, 1], wspace = 0.15, subplot_spec = main_gs[0, 0])

ax1 = fig.add_subplot(top_gs[0, 0])
ax2 = fig.add_subplot(top_gs[0, 1])
ax3 = fig.add_subplot(top_gs[0, 2])
ax4 = fig.add_subplot(top_gs[0, 3])
ax5 = fig.add_subplot(top_gs[0, 4])
cbar_ax = fig.add_subplot(main_gs[0, 1])

bottom_gs = gridspec.GridSpecFromSubplotSpec(1, 2, width_ratios = [1, 1], wspace = 0.2, subplot_spec = main_gs[1, 0])
ax6 = fig.add_subplot(bottom_gs[0, 0])
ax7 = fig.add_subplot(bottom_gs[0, 1])

vmin = 2e-4 #1e-4
vmax = 5e-1 #1e0
cmap = cmocean.cm.matter_r
norm = LogNorm(vmin = vmin, vmax = vmax)

# Plot
ax = ax1
ds = p5_noBS_KE.interp_like(p25_SQGBS_KE)
ax.pcolormesh(ds.xh, ds.yh, ds, cmap = cmap, norm = norm)
ax.set_facecolor((0.92, 0.92, 0.92))
ax.set_title(f'(a) p5noBS', fontsize = fontsize)
ax.set_xlabel('Longitude [°E]', fontsize = fontsize - 2)
ax.set_ylabel('Latitude [°N]', fontsize = fontsize - 2)
ax.tick_params(labelsize = fontsize - 2)
ax.set_xticks(np.linspace(20, 40, 2))

ax = ax2
ds = p25_noBS_KE
ax.pcolormesh(ds.xh, ds.yh, ds, cmap = cmap, norm = norm)
ax.set_facecolor((0.92, 0.92, 0.92))
ax.set_title(f'(b) p25noBS', fontsize = fontsize)
ax.set_xlabel('Longitude [°E]', fontsize = fontsize - 2)
ax.set_ylabel('', fontsize = fontsize - 2)
ax.tick_params(labelsize = fontsize - 2)
ax.set_xticks(np.linspace(20, 40, 2))

ax = ax3
ds = p5_SQGBS_KE.interp_like(p25_SQGBS_KE)
ax.pcolormesh(ds.xh, ds.yh, ds, cmap = cmap, norm = norm)
ax.set_facecolor((0.92, 0.92, 0.92))
ax.set_title(f'(c) p5BS', fontsize = fontsize)
ax.set_xlabel('Longitude [°E]', fontsize = fontsize - 2)
ax.set_ylabel('', fontsize = fontsize - 2)
ax.tick_params(labelsize = fontsize - 2)
ax.set_xticks(np.linspace(20, 40, 2))

ax = ax4
ds = p25_SQGBS_KE
ax.pcolormesh(ds.xh, ds.yh, ds, cmap = cmap, norm = norm)
ax.set_facecolor((0.92, 0.92, 0.92))
ax.set_title(f'(d) p25BS', fontsize = fontsize)
ax.set_xlabel('Longitude [°E]', fontsize = fontsize - 2)
ax.set_ylabel('', fontsize = fontsize - 2)
ax.tick_params(labelsize = fontsize - 2)
ax.set_xticks(np.linspace(20, 40, 2))

ax = ax5
ds = p03125_KE
plotted = ax.pcolormesh(ds.xh, ds.yh, ds, cmap = cmap, norm = norm)
ax.set_facecolor((0.92, 0.92, 0.92))
ax.set_title(f'(e) ref', fontsize = fontsize)
ax.set_xlabel('Longitude [°E]', fontsize = fontsize - 2)
ax.set_ylabel('', fontsize = fontsize - 2)
ax.tick_params(labelsize = fontsize - 2)
ax.set_xticks(np.linspace(20, 40, 2))

# # Add colorbar to the last column
cbar = fig.colorbar(plotted, cax = cbar_ax, extend = 'both')
cbar.ax.tick_params(labelsize = fontsize - 1)
cbar.set_label('EKE [m$^2$ s$^{-2}$]', loc = 'center', fontsize = fontsize, labelpad = 4)

bbox = cbar_ax.get_position()
cbar_ax.set_position([
    bbox.x0,   # move left
    bbox.y0,          # keep bottom the same
    bbox.width,  # make it fatter
    bbox.height       # keep height the same
])

# Hide tick labels
ax2.tick_params(labelleft = False)
ax3.tick_params(labelleft = False)
ax4.tick_params(labelleft = False)
ax5.tick_params(labelleft = False)

ax = ax6
p5_noBS_EKE_zm.plot(ax = ax, color = colorlist_p5[0], label = 'p5noBS', linewidth = linewidth)
p25_noBS_EKE_zm.plot(ax = ax, color = colorlist_p25[0], label = 'p25noBS', linewidth = linewidth)
p5_SQGBS_EKE_zm.plot(ax = ax, color = colorlist_p5[1], label = 'p5BS', linewidth = linewidth)
p25_SQGBS_EKE_zm.plot(ax = ax, color = colorlist_p25[1], label = 'p25BS', linewidth = linewidth)
p03125_EKE_zm.plot(ax = ax, color = colorlist_p03125[2], label = 'ref', linewidth = linewidth)
ax.grid()
ax.set_xlim([-65, 65])
ax.set_ylim([0, 0.0595])
ax.set_yticks(np.linspace(0, 0.05, 6))
ax.set_title('(f) EKE', fontsize = fontsize)
ax.set_xlabel('Latitude [°N]', fontsize = fontsize - 2)
ax.set_ylabel('[m$^2$ s$^{-2}$]', fontsize = fontsize - 2)
ax.tick_params(labelsize = fontsize - 2)
ax.legend(ncol = 3, loc = 'upper center', fontsize = fontsize - 2)

ax = ax7
((p5_dx * p5_static['area_t']).sum('xh') / p5_static['area_t'].sum('xh') / 1e3).plot(ax = ax, color = 'k', linewidth = linewidth - 0.5, linestyle = '--', label = '1/2° grid')
((p25_dx * p25_static['area_t']).sum('xh') / p25_static['area_t'].sum('xh') / 1e3).plot(ax = ax, color = 'gray', linewidth = linewidth - 0.5, linestyle = '--', label = '1/4° grid')

((p5_noBS_le * p5_static['area_t']).sum('xh') / p5_static['area_t'].sum('xh') / 1e3).plot(ax = ax, color = colorlist_p5[0], linewidth = linewidth)
((p25_noBS_le * p25_static['area_t']).sum('xh') / p25_static['area_t'].sum('xh') / 1e3).plot(ax = ax, color = colorlist_p25[0], linewidth = linewidth)

((p5_SQGBS_le * p5_static['area_t']).sum('xh') / p5_static['area_t'].sum('xh') / 1e3).plot(ax = ax, color = colorlist_p5[1], linewidth = linewidth)
((p25_SQGBS_le * p25_static['area_t']).sum('xh') / p25_static['area_t'].sum('xh') / 1e3).plot(ax = ax, color = colorlist_p25[1], linewidth = linewidth)

(p03125_le.mean('xh') / 1e3).plot(ax = ax, color = colorlist_p03125[2], linewidth = linewidth)
ax.plot(np.zeros(1), np.zeros([1]), color='w', alpha=0)#, label=' ')
ax.grid()
ax.set_xlim([-65, 65])
ax.set_ylim([0, 250])
ax.set_yticks(np.linspace(0, 250, 6))
ax.set_title('(g) Energy-containing scale', fontsize = fontsize)
ax.set_xlabel('Latitude [°N]', fontsize = fontsize - 2)
ax.set_ylabel('[km]', fontsize = fontsize - 2)
ax.tick_params(labelsize = fontsize - 2)
ax.legend(ncol = 1, loc = 'upper right', fontsize = fontsize - 2)

plt.savefig('Fig_depth_ave_KE_energy_scale.png', bbox_inches = 'tight', dpi = 300)